In [1]:
##Cormac McHale

In [2]:
##Training the Neural Network

In [3]:
import gzip
import sklearn.preprocessing as pre
import keras as kr
import numpy as np
from keras.models import load_model

Using TensorFlow backend.


In [4]:
# Start a neural network, building it by layers.
model = kr.models.Sequential()

# Add a hidden layer with 1000 neurons and an input layer with 784.
model.add(kr.layers.Dense(units=1000, activation='relu', input_dim=784))
model.add(kr.layers.Dense(units=500, activation='sigmoid'))
model.add(kr.layers.Dense(units=1000, activation='relu'))
# Add a 10 neuron output layer for each digit??
model.add(kr.layers.Dense(units=10, activation='softmax'))

# Build the graph.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
##file reading and formatting
imagesFile  = open('MNIST/train-images.idx3-ubyte', 'rb')
labelsFile = open('MNIST/train-labels.idx1-ubyte', 'rb')
images = imagesFile.read()
labels = labelsFile.read()
images =  (~np.array(list(images[16:])).reshape(60000, 28, 28).astype(np.uint8)-255)
labels =  np.array(list(labels[ 8:])).astype(np.uint8)

In [6]:
inputs = images.reshape(60000, 784)
inputs[inputs>0] = 255

In [7]:
encoder = pre.LabelBinarizer()
encoder.fit(labels)
outputs = encoder.transform(labels)
print(inputs[0])
#print (inputs[0].size)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0 255 255 255 255 255 255 255 255 255 255
 255 255   0   0   0   0   0   0   0   0   0   0   0   0 255 255 255 255
 255 255 255 255 255 255 255 255 255 255 255 255   0   0   0   0   0   0
   0   0   0   0   0 255 255 255 255 255 255 255 255 255 255 255 255 255
 255 255 255   0   0   0   0   0   0   0   0   0   0   0   0 255 255 255
 255 255 255 255 255 255 255 255   0   0   0   0   

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dense_2 (Dense)              (None, 500)               500500    
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              501000    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                10010     
Total params: 1,796,510
Trainable params: 1,796,510
Non-trainable params: 0
_________________________________________________________________


In [9]:
##for a later consideration
model.fit(inputs, outputs, epochs=5, batch_size=100)

Epoch 1/5
60000/60000 [==============================] - 22s 367us/step - loss: 0.6253 - accuracy: 0.8014
Epoch 2/5
60000/60000 [==============================] - 22s 361us/step - loss: 0.4871 - accuracy: 0.8482
Epoch 3/5
60000/60000 [==============================] - 22s 362us/step - loss: 0.4436 - accuracy: 0.8594
Epoch 4/5
60000/60000 [==============================] - 22s 360us/step - loss: 0.4160 - accuracy: 0.8687
Epoch 5/5
60000/60000 [==============================] - 22s 362us/step - loss: 0.4302 - accuracy: 0.8639


In [10]:
##need to then use the test images on the Neural Network...
imagesTest  = open('MNIST/t10k-images.idx3-ubyte', 'rb')
labelsTest = open('MNIST/t10k-labels.idx1-ubyte', 'rb')
testImages = imagesTest.read()
testLabels = labelsTest.read()

In [11]:
type(testImages)

bytes

In [12]:
testImages[0:4]

b'\x00\x00\x08\x03'

In [13]:
testImages = (~np.array(list(testImages[16:])).reshape(10000, 28, 28).astype(np.uint8)-255)
testLabels =  np.array(list(testLabels[ 8:])).astype(np.uint8)
testInputs = testImages.reshape(10000, 784)
encoder = pre.LabelBinarizer()
encoder.fit(testLabels)
testOutputs = encoder.transform(testLabels)

In [14]:
# Have the network predict the classes of the test inputs.
predictions = model.predict(testInputs)
predictions = encoder.inverse_transform(predictions)
##Comparison
predictions == encoder.inverse_transform(testOutputs)
#check how 
(predictions == encoder.inverse_transform(testOutputs)).sum()

6522

In [15]:
#save the model
model.save('savedModel/my_model.h5')
##Initial Model finished... flask Next.. will return to this later
testInputs = testImages.reshape(10000, 784)
model.predict(testInputs)

array([[4.5439165e-07, 5.9285636e-08, 1.6134543e-05, ..., 9.9968421e-01,
        4.5176412e-07, 1.2346936e-04],
       [1.8184677e-05, 1.2025261e-02, 9.6622479e-01, ..., 2.1683156e-04,
        8.1102765e-04, 4.2260001e-07],
       [1.1271973e-06, 9.4232184e-01, 4.6781002e-04, ..., 2.3132699e-05,
        1.6450314e-02, 3.6503195e-06],
       ...,
       [9.6811531e-03, 9.4970991e-04, 3.4668557e-02, ..., 2.0325261e-01,
        7.0208453e-02, 6.1687225e-01],
       [7.9483334e-03, 4.5714574e-04, 9.2105074e-05, ..., 5.8143312e-05,
        3.1450579e-01, 4.2807232e-03],
       [5.1405476e-03, 1.1749270e-02, 4.6425420e-01, ..., 4.2195305e-05,
        4.9882494e-02, 1.4051504e-04]], dtype=float32)

In [16]:
##Model complete